### En este notebook utilizaremos el conjunto de validación para afinar el modelo

In [1]:
import json
from glob import glob
import spacy
from spacy.lang.es import Spanish
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy
from medspacy.target_matcher import TargetMatcher, TargetRule

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ResourceWarning)

Vemos que tal funciona el modelo en el set de Validación

In [3]:
read_files = glob("./data/dev/text_files/*.txt")
             
with open("./data/dev/allfiles_dev.txt", "w", encoding="utf-8" ) as outfile:
    for f in read_files:
        with open(f, "r", encoding="utf-8") as infile:
            txt = "{}\n{}".format(f, infile.read())
            txt = txt.replace("./data/dev/text_files\\", "")
            outfile.write(txt + "\n")

In [4]:
with open ("./data/dev/allfiles_dev.txt", "r", encoding="utf-8") as f:
    archivos_val = f.read()#.split("\n")
#archivos_val = str(archivos_val) # Lo transformamos en una cadena
#print (archivos_val) 

Cargamos el modelo entrenado

In [5]:
nlp = spacy.load("./models/model_antecedentes_ner_es_md")
nlp.pipe_names

['tagger', 'parser', 'entity_ruler', 'ner']

In [6]:
target_matcher = TargetMatcher(nlp)
Span.set_extension("CIE10", default="", force=True)
rules = TargetRule.from_json("./data/cie10_rules.jsonl")
target_matcher.add(rules)
nlp.add_pipe(target_matcher)
nlp.pipe_names

['tagger', 'parser', 'entity_ruler', 'ner', 'target_matcher']

In [7]:
nuevo_informe = nlp(archivos_val)
ante_encontrados = [[ent.text, ent._.CIE10] for ent in nuevo_informe.ents]

print (f"Número de antecedentes encontrados: {len(ante_encontrados)}\n")

# Visualizamos los antecedentes encontrados en el informe
#colors = {"ANTEC": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
#options = {"ents": ["ANTEC"], "colors": colors}
#displacy.render(nuevo_informe, style="ent", options=options)

# Guardamos los antecedentes en un archivo
#with open ('data/antecedentes_urgencias_Ner_Matcher_2_ES_MD.jsonl', "w", encoding="utf-8") as f:
    #json.dump(ante_encontrados, f, indent=1)

Número de antecedentes encontrados: 529



Vamos a ver como se comporta el modelo desabilitando componentes y así elegir la mejor combinación

In [8]:
nlp.disable_pipes('entity_ruler')
nlp.pipe_names

['tagger', 'parser', 'ner', 'target_matcher']

In [9]:
nuevo_informe = nlp(archivos_val)
ante_encontrados = [[ent.text, ent._.CIE10] for ent in nuevo_informe.ents]

print (f"Número de antecedentes encontrados: {len(ante_encontrados)}\n")

Número de antecedentes encontrados: 529



In [10]:
nlp.disable_pipes('target_matcher')
nlp.pipe_names

['tagger', 'parser', 'ner']

In [11]:
nuevo_informe = nlp(archivos_val)
ante_encontrados = [[ent.text, ent._.CIE10] for ent in nuevo_informe.ents]

print (f"Número de antecedentes encontrados: {len(ante_encontrados)}\n")

Número de antecedentes encontrados: 509



In [12]:
nlp = spacy.load("./models/model_antecedentes_ner_es_md")
nlp.pipe_names

['tagger', 'parser', 'entity_ruler', 'ner']

In [13]:
target_matcher = TargetMatcher(nlp)
Span.set_extension("CIE10", default="", force=True)
rules = TargetRule.from_json("./data/cie10_rules.jsonl")
target_matcher.add(rules)
nlp.add_pipe(target_matcher)
nlp.pipe_names

['tagger', 'parser', 'entity_ruler', 'ner', 'target_matcher']

In [14]:
nlp.disable_pipes('ner')
nlp.pipe_names

['tagger', 'parser', 'entity_ruler', 'target_matcher']

In [15]:
nuevo_informe = nlp(archivos_val)
ante_encontrados = [[ent.text, ent._.CIE10] for ent in nuevo_informe.ents]

print (f"Número de antecedentes encontrados: {len(ante_encontrados)}\n")

Número de antecedentes encontrados: 440



El mejor resultado se obtiene sin desabilitar ningún componente